# 🔬 IRH v68 → v69 Multi-Agent Verification Pipeline

**Purpose:** Comprehensive AI-assisted verification and correction of Intrinsic Resonance Holography v68 manuscript.

## Three-Agent Architecture

| Agent | Role | Output |
|-------|------|--------|
| **Agent 1: Analyzer** | Reads v68 at every checkpoint, identifies deficits with exact locations | Structured deficit report (JSON) |
| **Agent 2: Resolution** | Receives deficit report, uses code execution for verified calculations, self-reviews | Verified resolutions with calculation logs |
| **Agent 3: Integration** | Takes resolutions + manuscript, writes seamlessly, pauses for user approval | IRHv69.md |

## Validation Tiers
- **Tier 1 (Precision):** Deviations < 0.1% from experimental values
- **Tier 2 (Accuracy):** Deviations < 1%
- **Tier 3 (Order of Magnitude):** Deviations < 10%

---

## 1. Environment Setup

In [ ]:
# Install required packages
!pip install -q google-generativeai numpy sympy scipy

import google.generativeai as genai
from google.colab import userdata, files
import json
import re
import numpy as np
from sympy import *
from dataclasses import dataclass, field
from typing import List, Dict, Optional, Tuple
from datetime import datetime
import time

# Configure Gemini API
GEMINI_API_KEY = userdata.get('GEMINI_API_KEY')
genai.configure(api_key=GEMINI_API_KEY)

print("✅ Environment configured successfully")
print(f"📅 Session started: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

## 2. Upload IRH v68 Manuscript

In [ ]:
# Upload the v68 manuscript
print("📄 Please upload IntrinsicResonanceHolography_v68.md")
uploaded = files.upload()

# Get the uploaded filename
MANUSCRIPT_FILENAME = list(uploaded.keys())[0]
MANUSCRIPT_CONTENT = uploaded[MANUSCRIPT_FILENAME].decode('utf-8')

print(f"\n✅ Loaded: {MANUSCRIPT_FILENAME}")
print(f"📊 Total characters: {len(MANUSCRIPT_CONTENT):,}")
print(f"📊 Total lines: {len(MANUSCRIPT_CONTENT.splitlines()):,}")

## 3. Manuscript Reader & Parser

Parses the v68 manuscript into a structured format with exact location tracking for sections, subsections, and line numbers.

In [ ]:
@dataclass
class ManuscriptSection:
    """Represents a section of the manuscript with location metadata."""
    section_num: str  # e.g., "I", "II", "IX"
    title: str
    start_line: int
    end_line: int
    content: str
    subsections: List['ManuscriptSubsection'] = field(default_factory=list)

@dataclass
class ManuscriptSubsection:
    """Represents a subsection with location metadata."""
    number: str  # e.g., "3.1", "9.2"
    title: str
    start_line: int
    end_line: int
    content: str


class ManuscriptReader:
    """Parses IRH manuscript into structured sections with exact line tracking."""
    
    # Chapter patterns for IRH manuscript
    CHAPTER_PATTERN = re.compile(
        r'^##\\s+(I{1,3}|IV|V|VI{0,3}|IX|X)\\.?\\s+(.+)$',
        re.MULTILINE
    )
    SUBSECTION_PATTERN = re.compile(
        r'^###\\s+(\\d+\\.\\d+)\\s+(.+)$',
        re.MULTILINE
    )
    
    def __init__(self, content: str):
        self.raw_content = content
        self.lines = content.splitlines()
        self.sections: List[ManuscriptSection] = []
        self._parse()
    
    def _parse(self):
        """Parse manuscript into sections and subsections."""
        # Find all chapter headings
        chapter_matches = []
        for i, line in enumerate(self.lines, start=1):
            match = self.CHAPTER_PATTERN.match(line)
            if match:
                chapter_matches.append((i, match.group(1), match.group(2)))
        
        # Create sections
        for idx, (line_num, num, title) in enumerate(chapter_matches):
            # Determine end line
            if idx + 1 < len(chapter_matches):
                end_line = chapter_matches[idx + 1][0] - 1
            else:
                end_line = len(self.lines)
            
            content = '\\n'.join(self.lines[line_num - 1:end_line])
            section = ManuscriptSection(
                section_num=num,
                title=title.strip(),
                start_line=line_num,
                end_line=end_line,
                content=content
            )
            
            # Parse subsections within this section
            section.subsections = self._parse_subsections(
                content, line_num, end_line
            )
            self.sections.append(section)
    
    def _parse_subsections(self, content: str, base_line: int, end_line: int) -> List[ManuscriptSubsection]:
        """Parse subsections within a section."""
        subsections = []
        section_lines = content.splitlines()
        
        sub_matches = []
        for i, line in enumerate(section_lines):
            match = self.SUBSECTION_PATTERN.match(line)
            if match:
                sub_matches.append((i, match.group(1), match.group(2)))
        
        for idx, (rel_line, num, title) in enumerate(sub_matches):
            abs_start = base_line + rel_line
            if idx + 1 < len(sub_matches):
                abs_end = base_line + sub_matches[idx + 1][0] - 1
            else:
                abs_end = end_line
            
            sub_content = '\\n'.join(self.lines[abs_start - 1:abs_end])
            subsections.append(ManuscriptSubsection(
                number=num,
                title=title.strip(),
                start_line=abs_start,
                end_line=abs_end,
                content=sub_content
            ))
        
        return subsections
    
    def get_section(self, section_num: str) -> Optional[ManuscriptSection]:
        """Get a specific section by Roman numeral."""
        for section in self.sections:
            if section.section_num == section_num:
                return section
        return None
    
    def get_lines(self, start: int, end: int) -> str:
        """Get specific line range (1-indexed, inclusive)."""
        return '\\n'.join(self.lines[start - 1:end])
    
    def get_context_around(self, line: int, context: int = 5) -> Tuple[int, int, str]:
        """Get context around a specific line."""
        start = max(1, line - context)
        end = min(len(self.lines), line + context)
        return start, end, self.get_lines(start, end)
    
    def summary(self) -> str:
        """Return a summary of the manuscript structure."""
        lines = ["📚 MANUSCRIPT STRUCTURE\\n" + "="*50]
        for sec in self.sections:
            lines.append(f"\\n## {sec.section_num}. {sec.title}")
            lines.append(f"   Lines: {sec.start_line}-{sec.end_line}")
            for sub in sec.subsections:
                lines.append(f"   ### {sub.number} {sub.title} (L{sub.start_line}-{sub.end_line})")
        return '\\n'.join(lines)


# Parse the manuscript
manuscript = ManuscriptReader(MANUSCRIPT_CONTENT)
print(manuscript.summary())

## 4. Deficit Report Data Structures

In [ ]:
@dataclass
class DeficitItem:
    """A single identified deficit in the manuscript."""
    id: str
    section: str
    subsection: Optional[str]
    line_range: Tuple[int, int]
    issue_type: str  # 'mathematical', 'physical', 'logical', 'notational', 'derivation'
    severity: str    # 'critical', 'major', 'minor'
    tier: int        # 1, 2, or 3
    current_solution: str
    why_faulty: str
    resolution_path: str
    requires_calculation: bool
    
    def to_dict(self) -> dict:
        return {
            'id': self.id,
            'section': self.section,
            'subsection': self.subsection,
            'line_range': list(self.line_range),
            'issue_type': self.issue_type,
            'severity': self.severity,
            'tier': self.tier,
            'current_solution': self.current_solution,
            'why_faulty': self.why_faulty,
            'resolution_path': self.resolution_path,
            'requires_calculation': self.requires_calculation
        }

@dataclass
class DeficitReport:
    """Complete deficit analysis report."""
    manuscript_version: str
    analysis_timestamp: str
    total_sections_analyzed: int
    deficits: List[DeficitItem] = field(default_factory=list)
    
    def add_deficit(self, item: DeficitItem):
        self.deficits.append(item)
    
    def get_by_severity(self, severity: str) -> List[DeficitItem]:
        return [d for d in self.deficits if d.severity == severity]
    
    def get_requiring_calculation(self) -> List[DeficitItem]:
        return [d for d in self.deficits if d.requires_calculation]
    
    def to_json(self) -> str:
        return json.dumps({
            'manuscript_version': self.manuscript_version,
            'analysis_timestamp': self.analysis_timestamp,
            'total_sections_analyzed': self.total_sections_analyzed,
            'summary': {
                'total_deficits': len(self.deficits),
                'critical': len(self.get_by_severity('critical')),
                'major': len(self.get_by_severity('major')),
                'minor': len(self.get_by_severity('minor')),
                'requiring_calculation': len(self.get_requiring_calculation())
            },
            'deficits': [d.to_dict() for d in self.deficits]
        }, indent=2)
    
    def display_summary(self):
        print("\\n" + "="*60)
        print("📋 DEFICIT REPORT SUMMARY")
        print("="*60)
        print(f"Manuscript: {self.manuscript_version}")
        print(f"Analyzed: {self.analysis_timestamp}")
        print(f"Sections analyzed: {self.total_sections_analyzed}")
        print(f"\\nTotal deficits: {len(self.deficits)}")
        print(f"  🔴 Critical: {len(self.get_by_severity('critical'))}")
        print(f"  🟠 Major: {len(self.get_by_severity('major'))}")
        print(f"  🟡 Minor: {len(self.get_by_severity('minor'))}")
        print(f"  🔢 Requiring calculation: {len(self.get_requiring_calculation())}")


@dataclass
class Resolution:
    """A verified resolution for a deficit."""
    deficit_id: str
    resolved_content: str
    calculation_log: Optional[str]
    verification_passed: bool
    self_review_notes: str
    
    def to_dict(self) -> dict:
        return {
            'deficit_id': self.deficit_id,
            'resolved_content': self.resolved_content,
            'calculation_log': self.calculation_log,
            'verification_passed': self.verification_passed,
            'self_review_notes': self.self_review_notes
        }

print("✅ Data structures defined")

---

# 🤖 AGENT 1: Analyzer

**Role:** Reads v68 at every checkpoint, produces detailed technical deficit report with exact locations.

**Constraints:**
- No ad hoc postulates in resolution paths
- No circular reasoning
- Must specify exact section, subsection, and line ranges
- Must explain WHY current solution is faulty

---

In [ ]:
class Agent1_Analyzer:
    """
    Agent 1: Constructive Analyzer
    
    Reads the manuscript at every checkpoint and produces detailed
    deficit reports with exact locations.
    """
    
    SYSTEM_INSTRUCTION = """
You are the **Constructive Analyzer** for the Intrinsic Resonance Holography (IRH) theory.

Your role is to perform rigorous analysis of the manuscript and identify deficits that require correction.

## Analysis Framework

For each section, you must:
1. **Mathematical Verification**: Check all equations, derivations, and numerical results
2. **Physical Interpretation**: Verify dimensional consistency and physical meaning
3. **Logical Flow**: Identify gaps in reasoning or circular arguments
4. **Notational Consistency**: Check symbol usage across the manuscript
5. **Comparison to Experiment**: Validate numerical predictions against known values

## Validation Tiers
- **Tier 1**: Predictions within 0.1% of experimental values (precision regime)
- **Tier 2**: Predictions within 1% of experimental values (accuracy regime)
- **Tier 3**: Predictions within 10% of experimental values (order of magnitude)

## Output Format

For each deficit found, provide a JSON object with:
```json
{
  \"id\": \"DEF-SEC-NNN\",
  \"section\": \"Roman numeral\",
  \"subsection\": \"X.Y or null\",
  \"line_range\": [start, end],
  \"issue_type\": \"mathematical|physical|logical|notational|derivation\",
  \"severity\": \"critical|major|minor\",
  \"tier\": 1|2|3,
  \"current_solution\": \"What the manuscript currently says\",
  \"why_faulty\": \"Detailed explanation of the problem\",
  \"resolution_path\": \"How to fix WITHOUT ad hoc postulates or circular reasoning\",
  \"requires_calculation\": true|false
}
```

## Critical Constraints

1. **NO AD HOC POSTULATES**: Resolution paths must derive from existing axioms
2. **NO CIRCULAR REASONING**: Cannot assume what you're trying to prove
3. **EXACT LOCATIONS**: Must specify precise line numbers
4. **CONSTRUCTIVE CRITICISM**: Focus on how to improve, not just what's wrong

Return your analysis as a JSON array of deficit objects.
"""
    
    def __init__(self, manuscript: ManuscriptReader):
        self.manuscript = manuscript
        self.model = genai.GenerativeModel(
            model_name='gemini-2.5-pro-preview-05-06',
            system_instruction=self.SYSTEM_INSTRUCTION
        )
        self.deficit_report = DeficitReport(
            manuscript_version='v68',
            analysis_timestamp=datetime.now().isoformat(),
            total_sections_analyzed=0
        )
    
    def analyze_section(self, section: ManuscriptSection) -> List[DeficitItem]:
        """Analyze a single section and return deficits found."""
        print(f"\\n🔬 Analyzing Section {section.section_num}: {section.title}")
        print(f"   Lines: {section.start_line}-{section.end_line}")
        
        # Re-read the full manuscript for context at each checkpoint
        full_context = f"""
=== FULL MANUSCRIPT (for reference) ===
{self.manuscript.raw_content}

=== SECTION TO ANALYZE ===
Section: {section.section_num}. {section.title}
Lines: {section.start_line} to {section.end_line}

{section.content}

=== TASK ===
Analyze this section for deficits. Return a JSON array of deficit objects.
If no deficits are found, return an empty array: []
"""
        
        try:
            response = self.model.generate_content(full_context)
            
            # Parse JSON from response
            text = response.text
            # Extract JSON array
            json_match = re.search(r'\\[.*\\]', text, re.DOTALL)
            if json_match:
                deficits_data = json.loads(json_match.group())
            else:
                print(f"   ℹ️ No deficits found in section {section.section_num}")
                return []
            
            deficits = []
            for d in deficits_data:
                item = DeficitItem(
                    id=d.get('id', f'DEF-{section.section_num}-{len(deficits)+1:03d}'),
                    section=d.get('section', section.section_num),
                    subsection=d.get('subsection'),
                    line_range=tuple(d.get('line_range', [section.start_line, section.end_line])),
                    issue_type=d.get('issue_type', 'unknown'),
                    severity=d.get('severity', 'minor'),
                    tier=d.get('tier', 3),
                    current_solution=d.get('current_solution', ''),
                    why_faulty=d.get('why_faulty', ''),
                    resolution_path=d.get('resolution_path', ''),
                    requires_calculation=d.get('requires_calculation', False)
                )
                deficits.append(item)
                self.deficit_report.add_deficit(item)
            
            print(f"   ✅ Found {len(deficits)} deficit(s)")
            return deficits
            
        except Exception as e:
            print(f"   ❌ Error analyzing section: {e}")
            return []
    
    def run_full_analysis(self) -> DeficitReport:
        """Run analysis on all sections."""
        print("\\n" + "="*60)
        print("🚀 AGENT 1: Starting Full Manuscript Analysis")
        print("="*60)
        
        for section in self.manuscript.sections:
            self.analyze_section(section)
            self.deficit_report.total_sections_analyzed += 1
            time.sleep(1)  # Rate limiting
        
        self.deficit_report.display_summary()
        return self.deficit_report


# Initialize Agent 1
agent1 = Agent1_Analyzer(manuscript)
print("✅ Agent 1 (Analyzer) initialized")

### Run Agent 1 Analysis

In [ ]:
# Run the full analysis
deficit_report = agent1.run_full_analysis()

# Save intermediate artifact
with open('deficit_report_v68.json', 'w') as f:
    f.write(deficit_report.to_json())

print("\\n📁 Saved: deficit_report_v68.json")

In [ ]:
# Display detailed deficits
print("\\n" + "="*60)
print("📋 DETAILED DEFICIT LIST")
print("="*60)

for i, d in enumerate(deficit_report.deficits, 1):
    severity_icon = {'critical': '🔴', 'major': '🟠', 'minor': '🟡'}.get(d.severity, '⚪')
    print(f"\\n{severity_icon} [{d.id}] {d.issue_type.upper()}")
    print(f"   Section: {d.section}" + (f" / {d.subsection}" if d.subsection else ""))
    print(f"   Lines: {d.line_range[0]}-{d.line_range[1]}")
    print(f"   Severity: {d.severity} | Tier: {d.tier}")
    print(f"   Current: {d.current_solution[:100]}..." if len(d.current_solution) > 100 else f"   Current: {d.current_solution}")
    print(f"   Problem: {d.why_faulty[:150]}..." if len(d.why_faulty) > 150 else f"   Problem: {d.why_faulty}")
    print(f"   Resolution: {d.resolution_path[:150]}..." if len(d.resolution_path) > 150 else f"   Resolution: {d.resolution_path}")
    if d.requires_calculation:
        print("   🔢 Requires verified calculation")

---

# 🔧 AGENT 2: Resolution Agent

**Role:** Receives manuscript + deficit report, uses code execution for verified calculations, self-reviews work.

**Capabilities:**
- Code execution for numerical verification
- Symbolic mathematics with SymPy
- Self-review before submitting resolutions

---

In [ ]:
class Agent2_Resolution:
    """
    Agent 2: Resolution Agent
    
    Receives deficit report, generates verified resolutions using
    code execution, and performs self-review.
    """
    
    SYSTEM_INSTRUCTION = """
You are the **Resolution Agent** for the Intrinsic Resonance Holography (IRH) theory.

Your role is to generate verified resolutions for identified deficits.

## Resolution Process

For each deficit:
1. **Understand Context**: Read the full manuscript section where the deficit occurs
2. **Verify the Problem**: Confirm the deficit analysis is correct
3. **Generate Resolution**: Create corrected content that fixes the issue
4. **Execute Calculations**: For mathematical deficits, provide Python code to verify
5. **Self-Review**: Check your resolution before submitting

## Code Execution Guidelines

When calculations are needed, provide executable Python code using:
- `numpy` for numerical computation
- `sympy` for symbolic mathematics
- `scipy` for scientific functions

## Output Format

For each resolution, provide:
```json
{
  \"deficit_id\": \"DEF-XXX-NNN\",
  \"resolved_content\": \"The corrected text/equations to replace the faulty content\",
  \"calculation_code\": \"Python code to verify (if applicable)\",
  \"calculation_result\": \"Expected output from the code\",
  \"self_review\": {
    \"mathematically_sound\": true|false,
    \"no_ad_hoc\": true|false,
    \"no_circular_reasoning\": true|false,
    \"dimensionally_consistent\": true|false,
    \"notes\": \"Any additional review notes\"
  }
}
```

## Critical Constraints

1. All calculations must be VERIFIED with actual code execution
2. No hand-wavy approximations without explicit justification
3. Maintain the manuscript's notation and style
4. Preserve the logical flow of the argument
"""

    def __init__(self, manuscript: ManuscriptReader, deficit_report: DeficitReport):
        self.manuscript = manuscript
        self.deficit_report = deficit_report
        self.model = genai.GenerativeModel(
            model_name='gemini-2.5-pro-preview-05-06',
            system_instruction=self.SYSTEM_INSTRUCTION,
            tools='code_execution'
        )
        self.resolutions: List[Resolution] = []
    
    def resolve_deficit(self, deficit: DeficitItem) -> Optional[Resolution]:
        """Generate a verified resolution for a single deficit."""
        print(f"\\n🔧 Resolving [{deficit.id}]: {deficit.issue_type}")
        print(f"   Section: {deficit.section}, Lines: {deficit.line_range}")
        
        # Get relevant manuscript context
        section = self.manuscript.get_section(deficit.section)
        if not section:
            print(f"   ❌ Could not find section {deficit.section}")
            return None
        
        # Get the specific lines
        context_start = max(1, deficit.line_range[0] - 10)
        context_end = min(len(self.manuscript.lines), deficit.line_range[1] + 10)
        local_context = self.manuscript.get_lines(context_start, context_end)
        
        prompt = f"""
=== FULL MANUSCRIPT (for reference) ===
{self.manuscript.raw_content}

=== DEFICIT TO RESOLVE ===
{json.dumps(deficit.to_dict(), indent=2)}

=== LOCAL CONTEXT (Lines {context_start}-{context_end}) ===
{local_context}

=== TASK ===
Generate a verified resolution for this deficit.
{"Use code execution to verify any calculations." if deficit.requires_calculation else ""}

Return your resolution as a JSON object.
"""
        
        try:
            response = self.model.generate_content(prompt)
            
            # Extract resolution from response
            text = response.text
            json_match = re.search(r'\\{.*?\\}', text, re.DOTALL)
            
            if json_match:
                resolution_data = json.loads(json_match.group())
            else:
                print(f"   ⚠️ Could not parse resolution JSON")
                # Use the raw response as resolution
                resolution_data = {
                    'resolved_content': text,
                    'self_review': {'notes': 'Auto-extracted from response'}
                }
            
            # Extract calculation log from code execution
            calc_log = None
            if hasattr(response, 'candidates') and response.candidates:
                for part in response.candidates[0].content.parts:
                    if hasattr(part, 'executable_code'):
                        calc_log = part.executable_code.code
                    if hasattr(part, 'code_execution_result'):
                        if calc_log:
                            calc_log += f"\\n\\n# Output:\\n{part.code_execution_result.output}"
            
            # Determine verification status
            self_review = resolution_data.get('self_review', {})
            verification_passed = all([
                self_review.get('mathematically_sound', True),
                self_review.get('no_ad_hoc', True),
                self_review.get('no_circular_reasoning', True),
                self_review.get('dimensionally_consistent', True)
            ])
            
            resolution = Resolution(
                deficit_id=deficit.id,
                resolved_content=resolution_data.get('resolved_content', ''),
                calculation_log=calc_log,
                verification_passed=verification_passed,
                self_review_notes=json.dumps(self_review)
            )
            
            self.resolutions.append(resolution)
            
            status = "✅ PASSED" if verification_passed else "⚠️ NEEDS REVIEW"
            print(f"   {status}")
            
            return resolution
            
        except Exception as e:
            print(f"   ❌ Error resolving deficit: {e}")
            return None
    
    def run_all_resolutions(self) -> List[Resolution]:
        """Resolve all deficits in the report."""
        print("\\n" + "="*60)
        print("🚀 AGENT 2: Starting Resolution Process")
        print("="*60)
        
        # Prioritize by severity
        sorted_deficits = sorted(
            self.deficit_report.deficits,
            key=lambda d: {'critical': 0, 'major': 1, 'minor': 2}.get(d.severity, 3)
        )
        
        # Configurable exponential backoff for pacing/API rate limiting
        import os
        base_delay = float(os.getenv("GEMINI_RATE_LIMIT_BASE_DELAY_SECONDS", "1.0"))
        max_delay = float(os.getenv("GEMINI_RATE_LIMIT_MAX_DELAY_SECONDS", "8.0"))
        current_delay = base_delay
        
        for deficit in sorted_deficits:
            self.resolve_deficit(deficit)
            # Exponential backoff between resolutions to help respect API rate limits
            if current_delay > 0:
                time.sleep(current_delay)
                current_delay = min(current_delay * 2, max_delay)
        
        # Summary
        passed = sum(1 for r in self.resolutions if r.verification_passed)
        print(f"\\n" + "="*60)
        print(f"📊 Resolution Summary: {passed}/{len(self.resolutions)} verified")
        print("="*60)
        
        return self.resolutions


# Initialize Agent 2 (requires deficit report from Agent 1)
print("✅ Agent 2 (Resolution) class defined")
print("   Run after Agent 1 completes analysis")

### Run Agent 2 Resolutions

In [ ]:
# Initialize and run Agent 2
agent2 = Agent2_Resolution(manuscript, deficit_report)
resolutions = agent2.run_all_resolutions()

# Save intermediate artifact
with open('resolutions_v68.json', 'w') as f:
    json.dump([r.to_dict() for r in resolutions], f, indent=2)

print("\\n📁 Saved: resolutions_v68.json")

In [ ]:
# Display resolution details
print("\\n" + "="*60)
print("📋 RESOLUTION DETAILS")
print("="*60)

for r in resolutions:
    status = "✅" if r.verification_passed else "⚠️"
    print(f"\\n{status} [{r.deficit_id}]")
    print(f"   Content preview: {r.resolved_content[:200]}..." if len(r.resolved_content) > 200 else f"   Content: {r.resolved_content}")
    if r.calculation_log:
        print(f"   📊 Calculation verified with code execution")
    print(f"   Self-review: {r.self_review_notes[:100]}..." if len(r.self_review_notes) > 100 else f"   Self-review: {r.self_review_notes}")

---

# 🔀 AGENT 3: Integration Agent

**Role:** Takes resolutions + report + manuscript, writes seamlessly into updated paper.

**Workflow:**
1. Apply resolutions to manuscript
2. Ensure seamless integration
3. **PAUSE for user approval**
4. Output IRHv69.md

---

In [ ]:
class Agent3_Integration:
    """
    Agent 3: Integration Agent
    
    Takes resolutions and integrates them seamlessly into the manuscript.
    Pauses for user approval before finalizing.
    """
    
    SYSTEM_INSTRUCTION = """
You are the **Integration Agent** for the Intrinsic Resonance Holography (IRH) theory.

Your role is to integrate verified resolutions seamlessly into the manuscript.

## Integration Guidelines

1. **Preserve Voice**: Maintain the manuscript's academic writing style
2. **Smooth Transitions**: Ensure edits flow naturally with surrounding text
3. **Notation Consistency**: Use the same symbols and conventions throughout
4. **Cross-References**: Update any references to changed content
5. **Version Clarity**: The output is v69, building on v68

## Output Format

Return the complete updated manuscript with all resolutions integrated.
Mark significant changes with HTML comments: <!-- v69: [description] -->

## Quality Checks

Before finalizing, verify:
- All deficits have been addressed
- No orphaned references
- Consistent equation numbering
- Proper section/subsection structure maintained
"""

    def __init__(self, manuscript: ManuscriptReader, deficit_report: DeficitReport, resolutions: List[Resolution]):
        self.manuscript = manuscript
        self.deficit_report = deficit_report
        self.resolutions = resolutions
        self.model = genai.GenerativeModel(
            model_name='gemini-2.5-pro-preview-05-06',
            system_instruction=self.SYSTEM_INSTRUCTION
        )
        self.updated_manuscript = None
        self.change_log = []
    
    def prepare_integration_prompt(self) -> str:
        """Prepare the prompt for integration."""
        resolution_summary = []
        for r in self.resolutions:
            # Find matching deficit
            deficit = next((d for d in self.deficit_report.deficits if d.id == r.deficit_id), None)
            if deficit:
                resolution_summary.append({
                    'deficit_id': r.deficit_id,
                    'section': deficit.section,
                    'subsection': deficit.subsection,
                    'line_range': deficit.line_range,
                    'original_issue': deficit.current_solution,
                    'resolved_content': r.resolved_content,
                    'verified': r.verification_passed
                })
        
        return f"""
=== ORIGINAL MANUSCRIPT (v68) ===
{self.manuscript.raw_content}

=== RESOLUTIONS TO INTEGRATE ===
{json.dumps(resolution_summary, indent=2)}

=== TASK ===
Integrate all resolutions into the manuscript to create v69.
Return the COMPLETE updated manuscript with all changes applied.
Use <!-- v69: description --> comments to mark significant changes.
"""
    
    def generate_updated_manuscript(self) -> str:
        """Generate the updated manuscript with all resolutions integrated."""
        print("\\n" + "="*60)
        print("🚀 AGENT 3: Starting Integration Process")
        print("="*60)
        
        print(f"\\n📝 Integrating {len(self.resolutions)} resolution(s)...")
        
        prompt = self.prepare_integration_prompt()
        
        try:
            response = self.model.generate_content(prompt)
            self.updated_manuscript = response.text
            
            # Log changes
            for r in self.resolutions:
                self.change_log.append({
                    'deficit_id': r.deficit_id,
                    'verified': r.verification_passed
                })
            
            print(f"\\n✅ Integration complete")
            print(f"   Original length: {len(self.manuscript.raw_content):,} chars")
            print(f"   Updated length: {len(self.updated_manuscript):,} chars")
            
            return self.updated_manuscript
            
        except Exception as e:
            print(f"\\n❌ Error during integration: {e}")
            return None
    
    def display_preview(self, lines: int = 100):
        """Display a preview of the updated manuscript."""
        if not self.updated_manuscript:
            print("⚠️ No updated manuscript available. Run generate_updated_manuscript() first.")
            return
        
        preview_lines = self.updated_manuscript.splitlines()[:lines]
        print("\\n" + "="*60)
        print(f"📄 PREVIEW (first {lines} lines)")
        print("="*60 + "\\n")
        print('\\n'.join(preview_lines))
        print("\\n" + "="*60)
        print(f"... [{len(self.updated_manuscript.splitlines()) - lines} more lines]")


# Initialize Agent 3 (requires resolutions from Agent 2)
print("✅ Agent 3 (Integration) class defined")
print("   Run after Agent 2 completes resolutions")

### Run Agent 3 Integration

In [ ]:
# Initialize and run Agent 3
agent3 = Agent3_Integration(manuscript, deficit_report, resolutions)
updated_manuscript = agent3.generate_updated_manuscript()

# Show preview
agent3.display_preview(lines=50)

---

# ⏸️ USER APPROVAL CHECKPOINT

**Before saving IRHv69.md, please review the changes and approve.**

---

In [ ]:
# Display change summary for user review
print("\\n" + "="*60)
print("📋 CHANGE SUMMARY FOR APPROVAL")
print("="*60)

print(f"\\n📊 Statistics:")
print(f"   Deficits identified: {len(deficit_report.deficits)}")
print(f"   Resolutions generated: {len(resolutions)}")
verified = sum(1 for r in resolutions if r.verification_passed)
print(f"   Verified resolutions: {verified}/{len(resolutions)}")

print(f"\\n📝 Changes by section:")
sections_changed = {}
for d in deficit_report.deficits:
    sections_changed[d.section] = sections_changed.get(d.section, 0) + 1
for sec, count in sorted(sections_changed.items()):
    print(f"   Section {sec}: {count} change(s)")

print(f"\\n⚠️ Unverified resolutions:")
unverified = [r for r in resolutions if not r.verification_passed]
if unverified:
    for r in unverified:
        print(f"   - {r.deficit_id}")
else:
    print("   None - all resolutions verified! ✅")

print("\\n" + "="*60)

In [ ]:
# USER APPROVAL CHECKPOINT
print("\\n" + "🔔"*20)
print("\\n⏸️  USER APPROVAL REQUIRED")
print("\\nReview the changes above. Do you want to save IRHv69.md?")
print("\\n" + "🔔"*20 + "\\n")

# Prompt for approval
approval = input("Type 'yes' to approve and save, or 'no' to cancel: ").strip().lower()

if approval == 'yes':
    APPROVED = True
    print("\\n✅ APPROVED - Proceeding to save IRHv69.md")
else:
    APPROVED = False
    print("\\n❌ NOT APPROVED - IRHv69.md will NOT be saved")
    print("   You can re-run the integration or make manual adjustments.")

---

# 💾 Export IRHv69.md

**Only runs if user approved in the previous cell.**

---

In [ ]:
if APPROVED:
    # Save the updated manuscript
    output_filename = 'IntrinsicResonanceHolography_v69.md'
    
    with open(output_filename, 'w', encoding='utf-8') as f:
        f.write(updated_manuscript)
    
    print(f"\\n✅ Saved: {output_filename}")
    print(f"   Size: {len(updated_manuscript):,} characters")
    print(f"   Lines: {len(updated_manuscript.splitlines()):,}")
    
    # Also save the full pipeline artifacts
    pipeline_summary = {
        'version': 'v68 → v69',
        'timestamp': datetime.now().isoformat(),
        'deficits_found': len(deficit_report.deficits),
        'resolutions_generated': len(resolutions),
        'resolutions_verified': verified,
        'sections_modified': list(sections_changed.keys()),
        'approved': True
    }
    
    with open('v69_pipeline_summary.json', 'w') as f:
        json.dump(pipeline_summary, f, indent=2)
    
    print(f"\\n📁 Pipeline summary saved: v69_pipeline_summary.json")
    
    # Download files
    print("\\n📥 Downloading files...")
    files.download(output_filename)
    files.download('deficit_report_v68.json')
    files.download('resolutions_v68.json')
    files.download('v69_pipeline_summary.json')
    
    print("\\n🎉 Pipeline complete! All files downloaded.")
else:
    print("\\n⏸️ Export skipped - user did not approve.")
    print("   Re-run the approval cell when ready.")

---

# 📊 Optional: Detailed Analysis Reports

Additional analysis and visualization tools.

---

In [ ]:
# Deficit distribution by type and severity
print("\\n" + "="*60)
print("📊 DEFICIT ANALYSIS")
print("="*60)

# By type
by_type = {}
for d in deficit_report.deficits:
    by_type[d.issue_type] = by_type.get(d.issue_type, 0) + 1

print("\\nBy Issue Type:")
for t, count in sorted(by_type.items(), key=lambda x: -x[1]):
    bar = '█' * count
    print(f"  {t:15} {bar} ({count})")

# By tier
by_tier = {}
for d in deficit_report.deficits:
    by_tier[d.tier] = by_tier.get(d.tier, 0) + 1

print("\\nBy Validation Tier:")
tier_labels = {1: 'Tier 1 (<0.1%)', 2: 'Tier 2 (<1%)', 3: 'Tier 3 (<10%)'}
for t in [1, 2, 3]:
    count = by_tier.get(t, 0)
    bar = '█' * count
    print(f"  {tier_labels[t]:15} {bar} ({count})")

In [ ]:
# Show calculation logs from verified resolutions
print("\\n" + "="*60)
print("🔢 CALCULATION LOGS")
print("="*60)

for r in resolutions:
    if r.calculation_log:
        print(f"\\n--- {r.deficit_id} ---")
        print(r.calculation_log[:500])
        if len(r.calculation_log) > 500:
            print(f"... [{len(r.calculation_log) - 500} more characters]")

---

## 📝 Session Notes

**Pipeline Version:** 1.0  
**Target Repository:** brandonmccraryresearch-cloud/IRHV24  
**Notebook for PR:** IRHv68_MultiAgent_Verification.ipynb

### Changelog
- v1.0: Initial three-agent architecture (Analyzer → Resolution → Integration)

---